# GEORGE

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD

from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from spuco.datasets.group_labeled_dataset_wrapper import GroupLabeledDatasetWrapper
from spuco.models import model_factory
from spuco.evaluate import Evaluator
from spuco.group_inference import GeorgeInference
from spuco.robust_train.group_balance_batch_erm import GroupBalanceBatchERM
from spuco.utils import Trainer, set_seed

device = torch.device("cuda")
set_seed(0)

In [9]:
# I'm using this as the default since this is what I see in the notebooks
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE # is this the default?
 
trainset = SpuCoMNIST(
    root="./data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    download=True
)
valset = SpuCoMNIST(
    root="./data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val",
    download=True
)
testset = SpuCoMNIST(
    root="./data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test",
    download=True
)
trainset.initialize()
testset.initialize()
valset.initialize()

In [10]:
train_source = trainset.load_data()
val_source   = valset.load_data()
test_source  = testset.load_data()

# Step 1

In [12]:
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=32,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=1e-2, momentum=0.9),
    device=device,
    verbose=True
)
trainer.train(num_epochs=5)

Epoch 0:   3%|█                               | 47/1501 [00:00<00:11, 131.56batch/s, accuracy=21.875%, loss=1.6]

 | Epoch 0 | Loss: 1.6115756034851074 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6203930377960205 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.61808180809021 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6209357976913452 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6147788763046265 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6116019487380981 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6084849834442139 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6079459190368652 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6164103746414185 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6171886920928955 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6182259321212769 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6186599731445312 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6024773120880127 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6099514961242676 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.619115948677063 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6324031352996826 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6056044101715088 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.61408054

Epoch 0:  10%|██▊                           | 143/1501 [00:00<00:04, 305.68batch/s, accuracy=65.625%, loss=1.56]

 | Epoch 0 | Loss: 1.5960102081298828 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.594313144683838 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6091065406799316 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6001816987991333 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.5973995923995972 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.5901176929473877 | Accuracy: 37.5%
 | Epoch 0 | Loss: 1.5991939306259155 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6107033491134644 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.592674970626831 | Accuracy: 40.625%
 | Epoch 0 | Loss: 1.5933797359466553 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.5914359092712402 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.600452184677124 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.593161702156067 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.613664984703064 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.598297119140625 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6054785251617432 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.5927809476852417 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.612371802330017 | 

Epoch 0:  17%|█████▏                         | 253/1501 [00:00<00:02, 423.47batch/s, accuracy=90.625%, loss=1.3]

...

 | Epoch 4 | Loss: 0.007514095865190029 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.007982755079865456 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.031670864671468735 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006522553041577339 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.0083613945171237 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006910695694386959 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.23580223321914673 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.008158710785210133 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006512338761240244 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.00643541757017374 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.20659777522087097 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.2015024572610855 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.23456937074661255 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.008143260143697262 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.00808852817863226 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008954224176704884 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008375918492674828 | Accuracy: 100.0%
 |

Epoch 4:  96%|███████████████████████████▉ | 1448/1501 [00:03<00:00, 496.72batch/s, accuracy=93.75%, loss=0.393]

 | Epoch 4 | Loss: 0.007158635184168816 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.00692237401381135 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008126385509967804 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006718469318002462 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.007307421416044235 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008981611579656601 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.19189931452274323 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.007957075722515583 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.20259343087673187 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.007980389520525932 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006980920676141977 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.007260453421622515 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.00761239230632782 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008381468243896961 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006632304284721613 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006695046089589596 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.006856533233076334 | Accuracy: 100.0%

Epoch 4: 100%|███████████████████████████| 1501/1501 [00:03<00:00, 412.07batch/s, accuracy=100.0%, loss=0.00599]

 | Epoch 4 | Loss: 0.3932779133319855 | Accuracy: 93.75%
 | Epoch 4 | Loss: 0.00768623361364007 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.21029898524284363 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.2201111614704132 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.008911612443625927 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.00809417013078928 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008130000904202461 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.009197287261486053 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.22252295911312103 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.011395758017897606 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.010211648419499397 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.00877825915813446 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.009420277550816536 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.009695248678326607 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.008766015991568565 | Accuracy: 100.0%
 | Epoch 4 | Loss: 0.18026353418827057 | Accuracy: 96.875%
 | Epoch 4 | Loss: 0.005993544589728117 | Accuracy: 100.0%


# Step 2

In [15]:
train_outputs = trainer.get_trainset_outputs(features=True)

george_infer = GeorgeInference(
    Z=train_outputs,
    class_labels=trainset.labels,
    device=device,
    max_clusters=trainset.num_classes,
    verbose=True
)
group_partition = george_infer.infer_groups()

Getting model outputs: 100%|██████████████████████████████████████████████| 751/751 [00:00<00:00, 959.72batch/s]
/home/raayan/.conda/envs/george/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Clustering class-wise: 100%|██████████████████████████████████████████████████████| 5/5 [00:24<00:00,  4.86s/it]


# Step 3

In [20]:
from spuco.robust_train.group_dro import GroupDRO

In [27]:
val_evaluator = Evaluator(
    testset=valset,
    group_partition=valset.group_partition,
    group_weights=valset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)

group_dro = GroupDRO(
    model=model,
    num_epochs=5,
    trainset=robust_trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    val_evaluator=val_evaluator,
    verbose=True
)
group_dro.train()


Epoch 0:   1%|▍                               | 10/751 [00:00<00:15, 46.70batch/s, accuracy=96.875%, loss=0.032]

 | Epoch 0 | Loss: 0.07678616791963577 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.024253422394394875 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.06534157693386078 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.03277839347720146 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.06603927910327911 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.025229444727301598 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.025914393365383148 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.029268339276313782 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.013658842071890831 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.02795269340276718 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.05886867642402649 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.011715348809957504 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.018598521128296852 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.018534572795033455 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.015947487205266953 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.01226414181292057 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.07773096859455109 | Accuracy: 9

Epoch 0:   4%|█▏                              | 29/751 [00:00<00:09, 75.30batch/s, accuracy=98.4375%, loss=0.26]

 | Epoch 0 | Loss: 0.03196655213832855 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.027158353477716446 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.39528995752334595 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.007746166083961725 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.03732161968946457 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.04327651858329773 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.00961747020483017 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.02493288926780224 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.05453749746084213 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.11380144208669662 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.012175306677818298 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.00797395221889019 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007114495616406202 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.11709454655647278 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.014949859119951725 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.012981596402823925 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.006197678856551647 | Accurac

Epoch 0:   7%|█▉                            | 49/751 [00:00<00:08, 81.35batch/s, accuracy=98.4375%, loss=0.0474]

...

Group (4, 2) Accuracy: 63.347457627118644


Evaluating group-wise accuracy:  96%|█████████████████████████████████████████▎ | 24/25 [00:05<00:00,  4.13it/s]

Group (4, 3) Accuracy: 63.983050847457626


Evaluating group-wise accuracy: 100%|███████████████████████████████████████████| 25/25 [00:05<00:00,  4.19it/s]

Group (4, 4) Accuracy: 64.54352441613588
Epoch 4:  Val Worst-Group Accuracy: 17.29490022172949
Best Val Worst-Group Accuracy: 17.29490022172949


In [28]:
evaluator = Evaluator(
    testset=testset,                           # Use your test dataset
    group_partition=testset.group_partition,   # Ensure the test set has group labels; if not, use inferred groups
    group_weights=testset.group_weights,       # Use group weights if available (for worst-group computation)
    batch_size=32,
    model=group_dro.best_model,                # Use the robustly trained model
    device=device,
    verbose=True
)
evaluator.evaluate()

print("Worst-group accuracy:", evaluator.worst_group_accuracy)
print("Average accuracy:", evaluator.average_accuracy)

Evaluating group-wise accuracy:   4%|█▊                                          | 1/25 [00:00<00:05,  4.33it/s]

Group (0, 0) Accuracy: 19.858156028368793


Evaluating group-wise accuracy:   8%|███▌                                        | 2/25 [00:00<00:05,  4.15it/s]

Group (0, 1) Accuracy: 20.33096926713948


Evaluating group-wise accuracy:  12%|█████▎                                      | 3/25 [00:00<00:05,  4.21it/s]

...

Group (4, 2) Accuracy: 64.48362720403023


Evaluating group-wise accuracy:  96%|█████████████████████████████████████████▎ | 24/25 [00:05<00:00,  4.40it/s]

Group (4, 3) Accuracy: 68.93939393939394


Evaluating group-wise accuracy: 100%|███████████████████████████████████████████| 25/25 [00:05<00:00,  4.30it/s]

Group (4, 4) Accuracy: 68.68686868686869
Worst-group accuracy: ((3, 2), 17.632241813602015)
Average accuracy: 33.22
